# Part 2

## Tutorial Part 2: Computer Vision with TransferLearning

**Transfer Learning** - storing knowledge gained while solving one problem and applying it to a different but related problem.
> [More information](https://en.wikipedia.org/wiki/Transfer_learning)

TensorFlow has good selection of pre-trained models that can be imported right in TensorFlow model. It is called [*TensorFlow Hub*](https://tfhub.dev).
As a compliment to main framework Google has published also additional package called [*TensorFlow Datasets*](https://www.tensorflow.org/datasets/catalog/overview#all_datasets) which has collection of the most popular datasets.

In this part of tutorial we will use [*The Standford Dogs dataset*](https://www.tensorflow.org/datasets/catalog/stanford_dogs) imported through TensorFlow datasets.

## Check if gpu is available and TF version

In [1]:
import tensorflow as tf
tf.__version__

'2.10.0'

In [2]:
!nvidia-smi

Mon Oct 24 19:20:03 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 512.78       Driver Version: 512.78       CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   81C    P8     6W /  N/A |      0MiB /  4096MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [17]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

Physical devices cannot be modified after being initialized


## Get data

In [18]:
import tensorflow_datasets as tfds

In [19]:
(train_data, test_data), ds_info = tfds.load(name='stanford_dogs',
                                             split=['train', 'test'],
                                             shuffle_files=True,
                                             as_supervised=True,
                                             with_info=True,
                                             batch_size=32)

In [20]:
ds_info

tfds.core.DatasetInfo(
    name='stanford_dogs',
    full_name='stanford_dogs/0.2.0',
    description="""
    The Stanford Dogs dataset contains images of 120 breeds of dogs from around
    the world. This dataset has been built using images and annotation from
    ImageNet for the task of fine-grained image categorization. There are
    20,580 images, out of which 12,000 are used for training and 8580 for
    testing. Class labels and bounding box annotations are provided
    for all the 12,000 images.
    """,
    homepage='http://vision.stanford.edu/aditya86/ImageNetDogs/main.html',
    data_path='C:\\Users\\misst\\tensorflow_datasets\\stanford_dogs\\0.2.0',
    file_format=tfrecord,
    download_size=778.12 MiB,
    dataset_size=744.72 MiB,
    features=FeaturesDict({
        'image': Image(shape=(None, None, 3), dtype=tf.uint8),
        'image/filename': Text(shape=(), dtype=tf.string),
        'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=120),
        'objects': Sequ

In [21]:
train_data, test_data

(<PrefetchDataset element_spec=(TensorSpec(shape=(None, None, None, 3), dtype=tf.uint8, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>,
 <PrefetchDataset element_spec=(TensorSpec(shape=(None, None, None, 3), dtype=tf.uint8, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>)

## Prepare data

### Batch and prefetch

In [22]:
import tensorflow as tf

In [23]:
# train_data = train_data.batch(32).prefetch(tf.data.AUTOTUNE)
# test_data = test_data.batch(32).prefetch(tf.data.AUTOTUNE)
# TFDS already batched dataset for us
train_data = train_data.prefetch(tf.data.AUTOTUNE)
test_data = test_data.prefetch(tf.data.AUTOTUNE)

**Note:**

* **Batch size** - the number of samples that are passed to the network at once.
![Training with large minibatches is bad for your heath](assets/tweet_2.png)
[Revisiting Small Batch Training for deep Neural Networks paper](https://arxiv.org/abs/1804.07612)
* **Prefetching** overlaps the preprocessing and model execution of a training step.
> On the step `s`, the input pipeline is reading the data for step `s+1`.
>
> `tf.data.AUTOTUNE` tunes value dynamically at runtime.

[More information](https://www.tensorflow.org/guide/data_performance)

### Data Augmentation

*Data Augmentation* is important concept against *overfitting* problem.
* **Data Augmentation** - a technique to increase the diversity of your training set by applying random (but realistic) transformations, such as image rotation.
> [More Information](https://www.tensorflow.org/tutorials/images/data_augmentation)
* **Overfiting** -  concept in data science, which occurs when a statistical model fit exactly against its training data
> [More Information](https://www.ibm.com/cloud/learn/overfitting)

In [24]:
# Build data augmentation layer
# Note: in TensorFlow models can be used as layers
from tensorflow.keras import layers

data_augmentation = tf.keras.models.Sequential([
    layers.RandomFlip(), # https://www.tensorflow.org/api_docs/python/tf/keras/layers/RandomFlip
    layers.RandomZoom(0.2), # https://www.tensorflow.org/api_docs/python/tf/keras/layers/RandomZoom
    layers.RandomRotation(0.2) # https://www.tensorflow.org/api_docs/python/tf/keras/layers/RandomRotation
], name='data_augmentation')

## Build model

### Get pretrained model

Links:
- [Documentation](https://www.tensorflow.org/api_docs/python/tf/keras/applications/efficientnet/EfficientNetB0)

- [TF Hub](https://tfhub.dev/google/collections/efficientnet/1)

In [25]:
base_model = tf.keras.applications.EfficientNetB0(include_top=False)
base_model.trainable = True # Unfreeze model's weights
for layer in base_model.layers[:-10]:
    layer.trainable = False # Unfreeze all layers except for last 10

### Callbacks
* `ModelCheckpoint` - saves model or model weights at some frequency.
> [More information](https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/ModelCheckpoint)
* `EarlyStopping` - stops training when a monitored metric has stopped improving.
> [More information](https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/EarlyStopping)

In [26]:
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath='model_checkpoints/checkpoint.ckpt',
                                                         save_weights_only=True,
                                                         save_best_only=True,
                                                         save_freq='epoch',
                                                         verbose=1)

In [27]:
early_stopping_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                           verbose=1,
                                                           restore_best_weights=True)

### Create a model with Functional API

In [28]:
inputs = tf.keras.layers.Input(shape=(None, None, 3), name='input_layer')
x = data_augmentation(inputs)
x = base_model(x)
x = tf.keras.layers.GlobalAveragePooling2D(name='global_average_pooling_layer')(x)
outputs = tf.keras.layers.Dense(120, activation='softmax', name='output_layer')(x)

In [29]:
model = tf.keras.models.Model(inputs, outputs, name='cv_model')
model.summary()

Model: "cv_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_layer (InputLayer)    [(None, None, None, 3)]   0         
                                                                 
 data_augmentation (Sequenti  (None, None, None, 3)    0         
 al)                                                             
                                                                 
 efficientnetb0 (Functional)  (None, None, None, 1280)  4049571  
                                                                 
 global_average_pooling_laye  (None, 1280)             0         
 r (GlobalAveragePooling2D)                                      
                                                                 
 output_layer (Dense)        (None, 120)               153720    
                                                                 
Total params: 4,203,291
Trainable params: 1,046,952
Non-tr

In [32]:
model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['accuracy'])

In [ ]:
model.fit(train_data,
          validation_data=test_data,
          validation_steps=int(0.2*len(test_data)),
          epochs=100,
          callbacks=[early_stopping_callback,
                     checkpoint_callback],
          verbose=1)